# Swarm Docker API - Things that can be done with SWARM with Docker API

- This uses the unix socket connection
- Be sure to mount the docker socket to ensure you can communicate with the service 
- Example:    -v /var/run/docker.sock:/var/run/docker.sock

## Get swarm information

In [3]:
## Get Swarm Info
$Response = curl --unix-socket /var/run/docker.sock http://localhost/swarm
$Response | ConvertFrom-JSON | FL

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1617  100  1617    0     0   394k      0 --:--:-- --:--:-- --:--:--  394k

ID                     : f5nus1c3ew3j23t73rpvqajw1
Version                : @{Index=40546}
CreatedAt              : 2021-02-02T20:05:35.224328893Z
UpdatedAt              : 2021-02-10T08:16:18.117394315Z
Spec                   : @{Name=default; Labels=; Orchestration=; Raft=; Dispatcher=; CAConfig=; 
                         TaskDefaults=; EncryptionConfig=}
TLSInfo                : @{TrustRoot=-----BEGIN CERTIFICATE-----
                         MIIBazCCARCgAwIBAgIUas7/ZWZeLffJVRj7w0UcyrchBmcwCgYIKoZIzj0EAwIw
                         EzERMA8GA1UEAxMIc3dhcm0tY2EwHhcNMjEwMjAyMjAwMTAwWhcNNDEwMTI4MjAw
                         MTAwWjATMREwDwYDVQQDEwhzd2FybS1jYTBZMBMGByqGSM49AgEG

## Get list of nodes

In [9]:
## Get Nodes
$Response = curl --unix-socket /var/run/docker.sock http://localhost/nodes
$Response | ConvertFrom-JSON | FL
$nodes = ($Response | ConvertFrom-JSON)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5663    0  5663    0     0  1106k      0 --:--:-- --:--:-- --:--:-- 1106k

ID            : 5j73yhgfnabz8wsuz1x1k44vl
Version       : @{Index=40721}
CreatedAt     : 2021-02-02T20:05:36.900346056Z
UpdatedAt     : 2021-02-10T18:41:37.643445185Z
Spec          : @{Labels=; Role=manager; Availability=active}
Description   : @{Hostname=unraid2-ds1; Platform=; Resources=; Engine=; TLSInfo=}
Status        : @{State=ready; Addr=10.60.0.32}
ManagerStatus : @{Reachability=reachable; Addr=10.60.0.32:2377}

ID            : l65gl9zdng25ewjzet5oaag00
Version       : @{Index=40726}
CreatedAt     : 2021-02-02T20:05:37.073158883Z
UpdatedAt     : 2021-02-10T18:41:46.93272375Z
Spec          : @{Labels=; Role=manager; Availability=active}
Description   : @{Hostname=pro

## Get specific Node details

In [13]:
## Get Node details
$nodes[0].ID
$Response = curl --unix-socket /var/run/docker.sock http://localhost/nodes/$($nodes[0].ID)
$Response | ConvertFrom-JSON | FL *

5j73yhgfnabz8wsuz1x1k44vl
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1884  100  1884    0     0   367k      0 --:--:-- --:--:-- --:--:--  459k

ID            : 5j73yhgfnabz8wsuz1x1k44vl
Version       : @{Index=40721}
CreatedAt     : 2021-02-02T20:05:36.900346056Z
UpdatedAt     : 2021-02-10T18:41:37.643445185Z
Spec          : @{Labels=; Role=manager; Availability=active}
Description   : @{Hostname=unraid2-ds1; Platform=; Resources=; Engine=; TLSInfo=}
Status        : @{State=ready; Addr=10.60.0.32}
ManagerStatus : @{Reachability=reachable; Addr=10.60.0.32:2377}




## Find all containers that match 'nginx' image

In [39]:
## Find all containers on any swarm node that match 'nginx'
$Response = curl --unix-socket /var/run/docker.sock 'http://localhost/containers/json?all=true'
$containers = ($Response | ConvertFrom-JSON)

## Filter and convert to table
$filter = @('Names','Image','Label')
$Containers | Where-Object {$_.Image -contains "nginx"} | FT

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 73432    0 73432    0     0  1749k      0 --:--:-- --:--:-- --:--:-- 1707k
100 73432    0 73432    0     0  1749k      0 --:--:-- --:--:-- --:--:-- 1707k

Id                                                               Names               Image ImageID
--                                                               -----               ----- ------- 
59f5e67917a98f48519804e8caedb59ab27684a26e0903e3f77f0c622125cbab {/nginx}            nginx sha256:…
592bd8a128da2f089658d43f6c1bf73d6bb2f6bbfac9cd96015947333f23a9a1 {/cranky_yalow}     nginx sha256:…
3584a28015be602f2c7a9eb9448527c05dfbb534d8df67ef3cff704ad95ba9a3 {/cranky_antonelli} nginx sha256:…



## Remove Container ID

In [50]:
## Identify Container to be removed
$removeContainerID = $containers[0]
$removeContainerID
$Response = curl -X DELETE --unix-socket /var/run/docker.sock "http://localhost/containers/$($removeContainerID.Id)"
$Response


Id              : 3584a28015be602f2c7a9eb9448527c05dfbb534d8df67ef3cff704ad95ba9a3
Names           : {/cranky_antonelli}
Image           : nginx
ImageID         : sha256:298ec0e28760b8eb1aad79711dc29c19041c61d7cf342dd1f445e91f30500549
Command         : /docker-entrypoint.sh nginx -g 'daemon off;'
Created         : 1612975858
Ports           : {}
Labels          : @{maintainer=NGINX Docker Maintainers <docker-maint@nginx.com>}
State           : created
Status          : Created
HostConfig      : @{NetworkMode=default}
NetworkSettings : @{Networks=}
Mounts          : {}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0



## Check if container exists now

In [51]:
## Find all containers on any swarm node that match 'nginx'
$Response = curl --unix-socket /var/run/docker.sock 'http://localhost/containers/json?all=true'
$containers = ($Response | ConvertFrom-JSON)

## Filter and convert to table
$filter = @('Names','Image','Label')
$Containers | Where-Object {$_.Image -contains "nginx"} | FT

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 71287    0 71287    0     0  1420k      0 --:--:-- --:--:-- --:--:-- 1420k
